In [4]:
# libraries and set up
import requests
import json
from datetime import datetime
import pprint

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from pandas import Series, DataFrame

# Import API key
from api_keys import nyt_api_key, news_api_key

billionaire_metadata_path = "data/forbes_billionaires_geo.csv"

In [5]:
#Constants
nyt_api_url                = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'
news_api_url_everything    = 'https://newsapi.org/v2/everything?'
news_api_url_top_headlines = "https://newsapi.org/v2/top-headlines/sources?"

In [6]:
# Read the billionaire data
billionaire_metadata = pd.read_csv(billionaire_metadata_path)
billionaire_df = pd.DataFrame(billionaire_metadata)

#display(billionaire_df)

In [22]:
def run_nyt_api(data_row):
    
    billionaire_name = data_row["Name"]

    query        = f"q={billionaire_name}"
    filter_query = "&fq=news_desk:(\"World\" \"Business\") AND source:(\"The New York Times\")" #no longer needed
    date_range   = "&begin_date=20210101&end_date=20211231"
    facets       = "&facet_fields=source&facet=true"
    api_key      = f"&api-key={nyt_api_key}"
    
    #nyt_query_url = f"{nyt_api_url}{query}{filter_query}{facets}{date_range}{api_key}"
    
    nyt_query_url = f"{nyt_api_url}{query}{facets}{date_range}{api_key}"
    response      = requests.get(nyt_query_url)
    status_code   = response.status_code
    
    
    if status_code == 200:   
        
        json_obj = response.json() 
        print(json_obj["response"])
        facets = json_obj["response"]["facets"]["source"]["terms"]
        #print(f"NYT Api results = {facets}")
        
    else:
        print("Ooops")   

In [23]:
def run_newsapi_api(data_row):
    
    #https://newsapi.org/docs/endpoints/everything
    
    billionaire_name = data_row["Name"]
    
    top_headlines = False
    url = None
    
    if(top_headlines):
        url = news_api_url_top_headlines
    else:
        url = news_api_url_everything
        
    query      = f"q={billionaire_name}"
    language   = "&language=en"
    date_range = "&from=2021-12-07&to=2022-01-06" #only get a months worth of articles
    sort_by    = "&sortBy=popularity"
    api_key    = f"&apikey={news_api_key}"
    
    billionaire_name = data_row["Name"]    
    query_url = f"{url}{query}{language}{date_range}{sort_by}{api_key}"
    
    print(query_url)

In [24]:
is_nyt_api = True #for testing

for idx,data_row in billionaire_df.iterrows():
    
    if is_nyt_api:
        run_nyt_api(data_row)
    else:
        run_newsapi_api(data_row)

    break #only generate 1 response for now

{'docs': [{'abstract': 'Starting in January, space tourists will not receive a participation trophy for flying to space. But everyone will be on the honor roll.', 'web_url': 'https://www.nytimes.com/2021/12/10/science/astronaut-wings-faa-bezos-musk.html', 'snippet': 'Starting in January, space tourists will not receive a participation trophy for flying to space. But everyone will be on the honor roll.', 'lead_paragraph': 'Jeff Bezos is officially an astronaut. Really — just ask the federal government.', 'print_section': 'A', 'print_page': '18', 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2021/12/10/science/10astronaut-wings1/merlin_191091735_516e6e3c-d988-4fd8-bdf4-bafcb78febf6-articleLarge.jpg', 'height': 400, 'width': 600, 'legacy': {'xlarge': 'images/2021/12/10/science/10astronaut-wings1/merlin_191091735_516e6e3c-d988-4fd8-bdf4-bafcb78febf6-articleLarge.jpg', 'xlargewidth': 600, 'xla

In [ ]:
## 2. Get Articles
#bezos_articles = requests.get(query_url).json()

#pprint.pprint(bezos_articles)